# Análise Exploratória de Dados Climáticos e de Uso do Solo

- Andressa Silva de Oliveira
- Fabrício Neri Lima
- Isabelle Moschini Murollo
- Vinicius Grando Eller

## Contextualização e Importância do Tema

No contexto atual, os eventos climáticos extremos têm se tornado mais frequentes e intensos, trazendo consequências devastadoras para a população, a biodiversidade e as economias mundiais. Essa tendência está relacionada às mudanças climáticas globais, as quais são influenciadas por fatores como desmatamento, urbanização crescente e emissões de gases de efeito estufa. Nesse sentido, a investigação da relação entre esses eventos e os locais onde eles ocorrem é de grande importância para compreender os impactos ambientais e socioeconômicos relacionados, permitindo o desenvolvimento de estratégias mais eficazes para a sua mitigação.

Sendo assim, este projeto explora a conexão entre eventos climáticos extremos e variáveis ambientais diversas, como aumento de áreas de plantio e degradação de florestas. Para isso, foram utilizadas as seguintes bases de dados: FAO (Land Cover and Forest Area), que fornece informações detalhadas sobre cobertura terrestre e áreas florestais, e EM-DAT, que documenta desastres naturais em escala global. Essas fontes são robustas e permitem avaliar padrões globais, além de realizar comparações regionais e históricas, contribuindo para a identificação de tendências e hotspots climáticos, os quais demandam maior atenção.

Dessa forma, objetiva-se, com este estudo, entender as causas e as consequências da interação entre fatores naturais e humanos e fornecer insights que possam vir a embasar políticas públicas e iniciativas sustentáveis. Ainda, a análise dessas relações é relevante nas circunstâncias atuais, em que a humanidade enfrenta desafios como o aumento das desigualdades climáticas e a necessidade de adoção de práticas ambientais mais responsáveis. Nesse sentido, este projeto busca, além de gerar insights de relevância ao contexto descrito, destacar a importância do uso da ciência de dados na geração de soluções para questões climáticas globais.


### Origem das Bases de Dados Utilizadas
Para o desenvolvimento das análises contidas neste projeto, utilizou-se duas bases de dados, sendo elas: FAO (Land Cover and Forest Area) e EM-DAT (Emergency Events Database). A primeira base contém dados anuais de 1992-2020 sobre o uso da terra, incluindo tipos de cobertura e índices que influenciam o clima, tendo sido publicada em 2022 pela FAOSTAT (Food and Agriculture Organization Statistics). A FAOSTAT é uma base de dados global da Organização das Nações Unidas para a Alimentação e a Agricultura (FAO) que oferece estatísticas agrícolas, alimentares e ambientais de países, desde 1961. 

Já a segunda base apresenta dados sobre o número e tipos de desastres naturais, agrupados anualmente, permitindo análise da frequência e intensidade de eventos climáticos ao longo do tempo, tendo sido publicada pelo EM-DAT (Emergency Events Database) em 2023. O EM-DAT é uma base global criada pelo Centro de Pesquisa sobre a Epidemiologia de Desastres (CRED), a qual registra e analisa dados sobre desastres naturais e tecnológicos.

## Montagem da base de dados

O script é responsável por processar e transformar arquivos CSV contendo dados brutos em um formato mais organizado e padronizado, utilizando **Dask** para lidar de forma eficiente com grandes volumes de dados. Ele começa identificando todos os arquivos `.csv` em uma pasta especificada (`FOLDER_PATH`) e analisa os nomes das colunas desses arquivos. Durante esse processo, as colunas são renomeadas para seguir um padrão uniforme, convertendo os nomes para letras minúsculas, removendo espaços e underscores. Caso alguma coluna esperada não esteja presente em um arquivo, ela é adicionada com valores nulos (`None`).

Além disso, o script separa as colunas em dois grupos: colunas gerais (como informações de identificação, por exemplo, `country`, `indicator`, etc.) e colunas específicas de anos (como `1990`, `2000`, etc.). Em seguida, ele transforma os dados de cada arquivo usando a técnica de "melt". Nesse processo, as colunas de anos são convertidas em linhas, criando duas novas colunas: uma chamada "year", que armazena o ano, e outra chamada "feature_value", que contém o valor da métrica correspondente. Uma nova coluna, chamada "datasource", também é adicionada para indicar de qual arquivo cada dado foi extraído.

Depois de normalizar e transformar os dados, o script os organiza em uma tabela final. Para isso, ele agrupa as informações por país, ano e outros identificadores, e pivota os dados de forma que cada métrica (indicador combinado com sua unidade) se torne uma coluna individual. Por fim, o script exporta os resultados em dois arquivos: um arquivo intermediário (`dataConcat_silver.csv`), que contém os dados após a normalização e transformação inicial, e um arquivo final (`dataConcat_gold.csv`), que apresenta os dados consolidados e prontos para análise. Em resumo, o script organiza dados brutos de múltiplas fontes, padroniza seus formatos e os consolida em um único conjunto de dados estruturado, ideal para análises futuras.

```
import glob
import os
import re
from typing import List, Tuple

import dask.dataframe as dd

FOLDER_PATH = "C:/Users/fabri/OneDrive/Area_de _trabalho/BigDataProject/data/raw"
OUTPUT_INTERIM = "data/interim/dataConcat_silver.csv"
OUTPUT_PROCESSED = "data/processed/dataConcat_gold.csv"

dtypes = {"ISO2": "object", "ObjectId": "int64"}


def get_csv_files(folder_path: str) -> List[str]:
    return glob.glob(os.path.join(folder_path, "*.csv"))


def get_final_columns(folder_path: str) -> Tuple[List[str], List[str]]:
    """
    Determines the final set of normalized column names and year columns
    from the CSV files in the given folder.

    Args:
        folder_path (str): Path to the folder containing CSV files.

    Returns:
        Tuple[List[str], List[str]]:
            - General (non-year) column names. eg: ctscode, ctsname, indicator...
            - Year-specific column names.
    """
    all_columns = set()

    for path in get_csv_files(folder_path):
        columns = dd.read_csv(path, dtype=dtypes, assume_missing=True).columns
        all_columns.update(columns)

    normalized_columns = {
        col.lower().strip().replace(" ", "").replace("_", "") for col in all_columns
    }
    year_columns = {col for col in normalized_columns if re.search(r"\d{4}$", col)}
    return list(normalized_columns - year_columns), list(year_columns)


def normalize_columns(df: dd.DataFrame, final_columns: List[str]) -> dd.DataFrame:
    """
    Normalizes the columns of a DataFrame by ensuring consistent naming
    and adding any missing columns with `None` values.

    Args:
        df (dd.DataFrame): Input DataFrame with raw column names.
        final_columns (List[str]): List of expected column names.

    Returns:
        dd.DataFrame: DataFrame with normalized column names.
    """
    df = df.rename(
        columns=lambda col: col.lower().strip().replace(" ", "").replace("_", "")
    )
    for col in final_columns:
        if col not in df.columns:
            df[col] = None
    return df[final_columns]


def groupby_country(interim_df: dd.DataFrame) -> dd.DataFrame:
    """
    Groups and pivots data by country, year, and indicator unit.

    Args:
        interim_df (dd.DataFrame): Input DataFrame with melted data.

    Returns:
        dd.DataFrame: Pivoted DataFrame with indicator units as columns.
    """
    interim_df["year"] = interim_df["year"].str.replace("f", "")
    interim_df["indicator_unit"] = (
        interim_df["indicator"].str.replace(" ", "")
        + "_"
        + interim_df["unit"].str.replace(" ", "")
    )

    index_cols = ["country", "iso2", "iso3", "year"]

    df_processed = interim_df.compute()
    return df_processed.pivot_table(
        index=index_cols,
        columns="indicator_unit",
        values="feature_value",
        aggfunc="first",
    ).reset_index()


def main():
    general_columns, year_columns = get_final_columns(FOLDER_PATH)
    final_columns = general_columns + year_columns
    df_list = []

    for path in get_csv_files(FOLDER_PATH):
        df = dd.read_csv(path, dtype=dtypes, assume_missing=True).set_index("ObjectId")
        df = df.map_partitions(normalize_columns, final_columns=final_columns)
        df_melted = df.melt(
            id_vars=general_columns,
            value_vars=year_columns,
            var_name="year",
            value_name="feature_value",
        )
        df_melted["datasource"] = os.path.basename(path)
        df_list.append(df_melted)

    data_interim = dd.concat(df_list)

    data_processed = groupby_country(data_interim)

    data_interim.to_csv(OUTPUT_INTERIM, index=False, single_file=True)
    data_processed.to_csv(OUTPUT_PROCESSED, index=False)


if __name__ == "__main__":
    main()

```



### Referências

Artigos que Utilizam a Base de Land Cover e Forest Area (FAO):
1. ROSAN, T. M. et al. A multi-data assessment of land use and land cover emissions from Brazil during 2000–2019. 2021 Environ. Res. Lett. 16 074004. Disponível em: <https://iopscience.iop.org/article/10.1088/1748-9326/ac08c3/meta>
2. DOOLEY, K. et al.. Over-reliance on land for carbon dioxide removal in net-zero climate pledges. Nature Communications (2024) 15:9118. 23 out. 2024. Disponível em: <https://www.nature.com/articles/s41467-024-53466-0>
3. REINER, F. et al. More than one quarter of Africa’s tree cover is found outside areas previously classified as forest. Nature Communications (2023) 14:2258. 29 mar. 2023. Disponível em: <https://www.nature.com/articles/s41467-023-37880-4>


Artigos que Utilizam a Base de Desastres Climáticos Extremos (EM-DAT):

1. NEWMAN, R.; NOY, I. The global costs of extreme weather that are attributable to climate change. Nature Communications (2023) 14:6103. 29 set. 2023. Disponível em: <https://www.nature.com/articles/s41467-023-41888-1>
2. DONG, C. Indo-Pacific regional extremes aggravated by changes in tropical weather patterns. Nature Geoscience. Volume 17 979–986. 4 out. 2024. Disponível em: <https://www.nature.com/articles/s41561-024-01537-8>
3. BALAIAN, S. K.; SANDERS, B. F.; QOMI, M. J. A. How urban form impacts flooding. Nature Communications (2024) 15:6911. 19 ago. 2024. Disponível em: <https://www.nature.com/articles/s41467-024-50347-4>


## Objetivo da Análise

Esta análise tem como objetivo explorar dados combinados de duas diferentes bases, de forma a identificar relações entre aspectos climáticos e territoriais e a corrência de desastres ambientais ao redor do mundo. Em especial, foram levantadas as seguintes perguntas iniciais, a serem respondidas:

- Qual país que mais teve desastres naturais nos últimos 5 anos?
- Quais os principais tipos de desastres naturais?
- Qual o ano em que aconteceu mais desastres?
- Há uma tendência de aumento no número de desastres ao longo dos anos?


# Import e Preparação dos Dados

Como primeiro passo, após os imports das bibliotecas necessárias, vamos realizar a importação da base de dados.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import linregress

Como primeiro passo, após os imports das bibliotecas necessárias, vamos realizar a importação da base de dados.

In [ ]:
data = pd.read_csv("data/processed/dataConcat_gold.csv")

In [ ]:
data.head(5)

Como primeira etapa da análise, vamos ver o resumo do dataset através do método info(). Um beneficio desse método é conseguir ver a tipagem de cada coluna, para necessidades como limpeza, e etc.

In [ ]:
data.info()  # Informações sobre colunas, tipos e valores nulos

Abaixo podemos perceber que temos uma quantidade de 13451 linhas e 31 features.

In [ ]:
data.shape  # Número de linhas e colunas

Quando fazemos um print do nome das colunas, podemos perceber que não necessariamente temos clareza do que significam. Por isso, vamos fazer um processo de renomeá-las

In [ ]:
data.columns

In [ ]:
# Lista de nomes das colunas antes da transformação
colunas_originais = [
    'country', 'iso2', 'iso3', 'year',
    'Artificialsurfaces(includingurbanandassociatedareas)_1000HA',
    'Carbonstocksinforests_Milliontonnes',
    'ClimateAlteringLandCoverIndex_Index',
    'Climaterelateddisastersfrequency,NumberofDisasters:Drought_Numberof',
    'Climaterelateddisastersfrequency,NumberofDisasters:Extremetemperature_Numberof',
    'Climaterelateddisastersfrequency,NumberofDisasters:Flood_Numberof',
    'Climaterelateddisastersfrequency,NumberofDisasters:Landslide_Numberof',
    'Climaterelateddisastersfrequency,NumberofDisasters:Storm_Numberof',
    'Climaterelateddisastersfrequency,NumberofDisasters:TOTAL_Numberof',
    'Climaterelateddisastersfrequency,NumberofDisasters:Wildfire_Numberof',
    'Forestarea_1000HA', 'Grassland_1000HA', 'Herbaceouscrops_1000HA',
    'Indexofcarbonstocksinforests_Index', 'Indexofforestextent_Index',
    'Inlandwaterbodies_1000HA', 'Landarea_1000HA', 'Mangroves_1000HA',
    'Permanentsnowandglaciers_1000HA', 'Shareofforestarea_Percent',
    'Shrub-coveredareas_1000HA',
    'Shrubsand/orherbaceousvegetation,aquaticorregularlyflooded_1000HA',
    'Sparselynaturalvegetatedareas_1000HA',
    'Temperaturechangewithrespecttoabaselineclimatology,correspondingtotheperiod1951-1980_DegreeCelsius',
    'Terrestrialbarrenland_1000HA', 'Tree-coveredareas_1000HA',
    'Woodycrops_1000HA'
]

In [ ]:

# Colunas com nomes mais legiveis
colunas_melhoradas = [
    'country', 'iso2', 'iso3', 'year',
    'artificial_surfaces_1000ha',
    'carbon_stocks_in_forests_million_tonnes',
    'climate_altering_land_cover_index',
    'disasters_drought_number',
    'disasters_extreme_temperature_number',
    'disasters_flood_number',
    'disasters_landslide_number',
    'disasters_storm_number',
    'disasters_total_number',
    'disasters_wildfire_number',
    'forest_area_1000ha', 'grassland_1000ha', 'herbaceous_crops_1000ha',
    'carbon_stocks_index', 'forest_extent_index',
    'inland_water_bodies_1000ha', 'land_area_1000ha', 'mangroves_1000ha',
    'permanent_snow_and_glaciers_1000ha', 'share_of_forest_area_percent',
    'shrub_covered_areas_1000ha',
    'flooded_shrubs_or_herbaceous_vegetation_1000ha',
    'sparsely_natural_vegetated_areas_1000ha',
    'temperature_change_since_1951_1980_celsius',
    'terrestrial_barren_land_1000ha', 'tree_covered_areas_1000ha',
    'woody_crops_1000ha'
]

In [ ]:
data.columns = colunas_melhoradas

O tratamento deixa a leitura mais legível de fácil compreensão, como pode ser visto abaixo:

In [ ]:
data.columns

In [ ]:
data.info()  # Informações sobre colunas, tipos e valores nulos

O describe() vai mostrar um resumo estatistico dos dados numericos presentes no dataframe.

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()  # Total de valores nulos por coluna

Pode ser tirada algumas hipóteses em relação ao comportamento dos dados, de maneira preliminar. Para os desastres naturais, por exemplo, é esperado que haja uma grande quantidade de dados faltantes. Na verdade, eles não são faltantes, mas a sua ausência apenas demonstra que não houve algum evento extremo naquele país durante aquele ano, o que é esperado, uma vez que eventos extremos deveriam ser pouco comuns.

In [ ]:
data.isnull().mean() * 100  

A hipótese pode ser verdade para dados como desastres climáticos intensos, mas quando olhamos para a porcentagem dos outros dados, parece haver um, excesso de dados faltantes. Pode ser necessário recortar temporalmente o dataset de modo a ter dados mais relevantes, Vamos agora analisar de maneira gráfica o comportamento dos dados e como eles estão distribuidos.

Primeiro, vamos preencher os valores faltantes das colunas de desastres climáticos com zeros.

In [ ]:
colunas_para_preencher = ['disasters_drought_number','disasters_extreme_temperature_number','disasters_flood_number','disasters_landslide_number',
'disasters_storm_number','disasters_total_number','disasters_wildfire_number']

data[colunas_para_preencher] = data[colunas_para_preencher].fillna(0)

In [ ]:
data.isnull().mean() * 100

## Histogramas para cada variável

In [ ]:
# Histogramas individuais com FacetGrid
import math

num_cols = data.select_dtypes(include=['float64', 'int']).columns  # Apenas colunas numéricas
num_plots = len(num_cols)
cols = 3  # Número de gráficos por linha
rows = math.ceil(num_plots / cols)

fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 5))  # Define o tamanho total
axes = axes.flatten()  # Achata a matriz de eixos

for i, col in enumerate(num_cols):
    sns.histplot(data[col], bins=20, ax=axes[i], kde=True)
    axes[i].set_title(col)
    
# Remove gráficos extras
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
fig.savefig('histogramas.png', dpi=300, bbox_inches='tight')  # Salva como imagem PNG com alta resolução
plt.show()


O registro histórico mostra como as variáveis estão distribuídas na estrutura de dados, e é possível pode ver algumas coisas interessantes. Primeiro, a maioria deles tem distribuição e comprimento de cauda assimétricos – ou seja, existem alguns valores que são maiores que a maioria, como acidente_número_total e área_floresta_1000ha. Além disso, existem variáveis que mostram claramente valores grandes, como carbono_stocks_in_forests_million_tonnes e terrestrial_barren_land_1000ha, que são mostradas em outras.
Por outro lado, algumas variáveis, como temperatura_mudança_desde_1961_1990_celsius, parecem ter uma distribuição mais "normal", que parece um padrão normal. Outros são muito sensíveis a valores próximos de zero, como número_acidente e manguezais_1000ha. Isto pode indicar que a maioria dos dados apresenta valores baixos, enquanto alguns são elevados.
Existem também variáveis que seguem padrões simples, como números, que representam medidas constantes ao longo do tempo. Outros, como part_of_the_barrel_forest, têm uma distribuição ampla e variada.

## Matriz de correlação 

In [ ]:
# Filtrar apenas colunas numéricas
corr = data.select_dtypes(include=['float64', 'int']).corr()

In [ ]:
plt.figure(figsize=(12, 10))  # Ajusta o tamanho do gráfico
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".1f")  # Reduz para 2 casas decimais
plt.xticks(rotation=45, ha='right')  # Rotaciona os rótulos do eixo X
plt.yticks(rotation=0)  # Mantém os rótulos do eixo Y horizontais
plt.title('Heatmap de Correlação', fontsize=16)  # Adiciona um título
plt.savefig('heatmap_correlacao.png', dpi=300, bbox_inches='tight')  # Salva com alta resolução
plt.show()

A matriz de correlação revela padrões importantes de serem pontuados sobre como as variáveis globais se relacionam entre si. Um dos destaques é a forte correlação positiva entre os estoques de carbono em florestas e a área de floresta, o que era esperado, já que a quantidade de carbono estocado está diretamente ligada à extensão dessas áreas. Outro ponto relevante é a correlação média a forte entre áreas urbanizadas, representadas por superfícies artificiais, e o número de desastres climáticos. Esse padrão sugere que a expansão das cidades pode estar associada a uma maior vulnerabilidade ou a impactos mais intensos de eventos climáticos extremos.
Além disso, observa-se uma relação negativa entre alterações climáticas e variáveis como estoques de carbono e índices florestais, reforçando a ideia de que mudanças no clima afetam negativamente os ecossistemas naturais, diminuindo sua capacidade de capturar e armazenar carbono. Existe uma correlação negativa entre a área de vegetação natural e a área de vegetação artificial, indicando que o crescimento urbano tende a ocorrer em áreas naturais.
Outro ponto interessante é a relação positiva entre o número de desastres naturais e as terras construídas pelo homem, o que faz com que as atividades humanas melhorem a relação entre elas, as mudanças no uso da terra e os efeitos das condições climáticas severas. Estas medidas destacam a ligação entre as condições ambientais e o uso humano da terra, enfatizando a importância de políticas para reduzir os impactos das alterações climáticas, especialmente nas áreas urbanas, e proteger a vida dos recursos naturais.

## Investigação dos dados faltantes

In [ ]:
faltantes_por_linha = data.isnull().sum(axis=1)
linhas_com_2_ou_mais_faltantes = data[faltantes_por_linha >= 2]
faltantes_por_ano = linhas_com_2_ou_mais_faltantes.groupby('year').size()

In [ ]:
faltantes_por_ano

In [ ]:
faltantes_por_ano.plot(kind='bar', figsize=(10, 6), color='orange')
plt.title('Linhas com 2 ou Mais Dados Faltantes por Ano', fontsize=16)
plt.xlabel('Ano', fontsize=14)
plt.ylabel('Quantidade de Linhas', fontsize=14)
plt.xticks(rotation=70)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
faltantes_por_linha = data.isnull().sum(axis=1)
linhas_com_15_ou_mais_faltantes = data[faltantes_por_linha >= 15]
faltantes_por_ano = linhas_com_15_ou_mais_faltantes.groupby('year').size()

In [ ]:
faltantes_por_ano.plot(kind='bar', figsize=(10, 6), color='orange')
plt.title('Linhas com 15 ou Mais Dados Faltantes por Ano', fontsize=16)
plt.xlabel('Ano', fontsize=14)
plt.ylabel('Quantidade de Linhas', fontsize=14)
plt.xticks(rotation=70)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Por conta do numero de dados faltantes nas colunas antes de 1992 e em 2023, foi decidido dropar as linhas com esses dados. Isso porque havia uma ausência muito grande de dados.

In [ ]:
# Conta o número de valores faltantes em cada linha
faltantes_por_linha = data.isnull().sum(axis=1)
# Filtra apenas as linhas com menos de 2 valores faltantes
new_data = data[faltantes_por_linha < 15]

# Exibindo o tamanho do dataset antes e depois
print(f"Tamanho original do dataset: {data.shape[0]} linhas")
print(f"Tamanho do dataset limpo: {new_data.shape[0]} linhas")

Antes de refazer as visualizações e correlações dos dados, agora que eles estão devidamente limpos e com a quantidade certa de linhas, pode ser interessante criar novas colunas pela derivação dos dados que ja temos disponivel

## Novas colunas a partir dos dados existentes

1. Proporção de Áreas Urbanas:

In [ ]:
new_data.loc[:, 'urban_area_percentage'] = (new_data['artificial_surfaces_1000ha'] / new_data['land_area_1000ha']) * 100

2. Proporção de Áreas Naturais:

In [ ]:
new_data.loc[:, 'forest_percentage'] = (new_data['forest_area_1000ha'] / new_data['land_area_1000ha']) * 100

3.  Taxa de variação do estoque de carbono


In [ ]:
new_data.loc[:, 'carbon_stock_change_rate'] = new_data.groupby('country')['carbon_stocks_in_forests_million_tonnes'].diff()

4. Mudança de temperatura acumulada (ano a ano)

In [ ]:
new_data.loc[:, 'cumulative_temp_change'] = new_data.groupby('country')['temperature_change_since_1951_1980_celsius'].cumsum()

5. Taxa de cobertura arbórea

In [ ]:
new_data.loc[:, 'tree_cover_percentage'] = (new_data['tree_covered_areas_1000ha'] / new_data['land_area_1000ha']) * 100

6. Taxa de variação do desmatamento (ano a ano)

In [ ]:
new_data.loc[:, 'deforestation_rate'] = new_data.groupby('country')['forest_area_1000ha'].diff() / new_data['land_area_1000ha'] * 100

Agora que temos novos indices, vamos averiguar como está o preenchimento das colunas.

In [ ]:
new_data.isnull().mean() * 100  # Percentual de valores nulos por coluna

In [ ]:
# Filtrar apenas colunas numéricas
new_corr = new_data.select_dtypes(include=['float64', 'int']).corr()

In [ ]:
plt.figure(figsize=(12, 10))  # Ajusta o tamanho do gráfico
sns.heatmap(new_corr, annot=True, cmap='coolwarm', fmt=".1f")  # Reduz para 2 casas decimais
plt.xticks(rotation=45, ha='right')  # Rotaciona os rótulos do eixo X
plt.yticks(rotation=0)  # Mantém os rótulos do eixo Y horizontais
plt.title('Heatmap de Correlação', fontsize=16)  # Adiciona um título
plt.show()

## Correlação por países

Uma pergunta que pode surgir, é: os dados se comportam igualmente (correlação) para os países? Levando isso em consideração, pode ser feito um heatmap para cada país, de modo a entender melhor os comportamentos. 

É esperado que os comportamentos ambientais e climáticos sejam diferentes em cada país, isso porque a incidência solar, chuvas, ventos e etc são diferentes em cada região do globo, além de aspectos humanos que afetam diretamente, como urbanização.

In [ ]:
countries = new_data['country'].unique()
len(countries)

In [ ]:
countries

Seria inviável analisar 232 heatmaps, por isso, vamos plotar apenas 4.

In [ ]:
# Configurar o número de colunas no layout
num_columns = 2  # Quantos heatmaps por linha
countries = new_data['country'].unique()[:4]  # Selecionar os primeiros 10 países para exemplificar

# Criar subplots
num_rows = (len(countries) + num_columns - 1) // num_columns  # Calcular o número de linhas
fig, axes = plt.subplots(num_rows, num_columns, figsize=(num_columns * 5, num_rows * 5))
axes = axes.flatten()  # Transformar a matriz de eixos em um vetor

# Iterar sobre os países e gerar os heatmaps
for i, country in enumerate(countries):
    # Filtrar os dados para o país atual
    country_data = new_data[new_data['country'] == country]

    # Selecionar apenas colunas numéricas
    numeric_columns = country_data.select_dtypes(include='number').columns
    country_numeric_data = country_data[numeric_columns]

    # Calcular a matriz de correlação
    country_corr = country_numeric_data.corr()

    # Plotar o heatmap no subplot correspondente
    sns.heatmap(
        country_corr, 
        annot=False, 
        cmap='coolwarm', 
        ax=axes[i], 
        cbar=False  # Ocultar a barra de cores individualmente
    )
    axes[i].set_title(country, fontsize=12)
    axes[i].axis('off')  # Opcional: remover os rótulos dos eixos

# Ajustar o layout e remover subplots vazios
for j in range(len(countries), len(axes)):
    fig.delaxes(axes[j])  # Remove os eixos não utilizados

plt.tight_layout()
plt.show()

Com o resultado obtido a partir dos 4 heatmaps, percebe-se que realmente há grandes variações na correlação de variáveis para cada país. Isso mostra que, para casos de predição, por exemplo, seria importante analisar individualmente cada país.

Uma questão identificada ao plotar esses heatmaps foi a disparidade na concentração de dados entre diferentes países, que contam com ausência de dados para variáveis diferentes para cada país.

## Perguntas iniciais para entendimento dos dados

### Qual país que mais teve desastres naturais nos últimos 5 anos?

In [ ]:
# Filtrar os últimos 5 anos (ajuste o ano atual conforme necessário)
latest_year = new_data['year'].max()
last_5_years = new_data[new_data['year'] >= (latest_year - 5)]

# Agrupar por país e somar o número total de desastres
disasters_by_country = last_5_years.groupby('country')['disasters_total_number'].sum()
most_disasters_country = disasters_by_country.idxmax()
most_disasters_count = disasters_by_country.max()

print(f"O país com mais desastres naturais nos últimos 5 anos foi {most_disasters_country}, com {most_disasters_count} desastres.")

### Quais os principais tipos de desastres naturais?


In [ ]:
# Selecionar colunas de desastres
disaster_columns = [
    'disasters_drought_number', 
    'disasters_extreme_temperature_number', 
    'disasters_flood_number', 
    'disasters_landslide_number', 
    'disasters_storm_number', 
    'disasters_wildfire_number'
]

# Somar ocorrências de cada tipo de desastre
disaster_totals = new_data[disaster_columns].sum().sort_values(ascending=False)

print("Principais tipos de desastres naturais:")
print(disaster_totals)

### Qual o ano em que aconteceu mais desastres?


In [ ]:
# Agrupar por ano e somar o número total de desastres
disasters_by_year = new_data.groupby('year')['disasters_total_number'].sum()
year_with_most_disasters = disasters_by_year.idxmax()
most_disasters_in_year = disasters_by_year.max()

print(f"O ano com mais desastres foi {year_with_most_disasters}, com {most_disasters_in_year} desastres.")

### Há uma tendência de aumento no número de desastres ao longo dos anos?


In [ ]:
#Agrupar por ano e calcular o número total de desastres
disasters_by_year = new_data.groupby('year')['disasters_total_number'].sum()

# Realizar a regressão linear
years = disasters_by_year.index
disasters = disasters_by_year.values
slope, intercept, r_value, p_value, std_err = linregress(years, disasters)

# Previsão da linha de tendência
trend_line = slope * years + intercept

# Exibir resultados
trend = "aumento" if slope > 0 else "diminuição"
print(f"A tendência é de {trend} no número de desastres ao longo dos anos.")
print(f"Coeficiente angular: {slope:.2f}, R²: {r_value**2:.2f}, p-value: {p_value:.4f}")

In [ ]:
# Criar o gráfico
plt.figure(figsize=(10, 6))
plt.scatter(years, disasters, color='blue', label='Número de Desastres (Dados)')
plt.plot(years, trend_line, color='red', linestyle='--', label=f'Tendência Linear (y={slope:.2f}x + {intercept:.2f})')
plt.title('Tendência do Número de Desastres ao Longo dos Anos')
plt.xlabel('Ano')
plt.ylabel('Número de Desastres')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

### Predição de comportamentos climáticos e ambientais no Brasil

A primeira etapa para analisar comportamentos do brasil e tentar predize-los envolve filtrar seus dados. Além disso, vamos plotar também suas correlações para discutir.

In [ ]:
# Filtrar os dados para o país atual
country_data = new_data[new_data['country'] == 'Brazil']
    
# Selecionar apenas colunas numéricas
numeric_columns = country_data.select_dtypes(include='number').columns
country_numeric_data = country_data[numeric_columns]
    
# Calcular a matriz de correlação
country_corr = country_numeric_data.corr()
    
# Plotar o heatmap de correlação para o país
plt.figure(figsize=(10, 10))
sns.heatmap(country_corr, annot=False, cmap='coolwarm')  # Remover números no gráfico
plt.title(f'Heatmap de Correlação - Brazil', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.show()

Com base na matriz e na utilidade das variáveis, existem variáveis que fazem mais sentido em serem previstas, como a mudança acumulada de temperatura, que mostra o quanto tá aumentando ou diminuindo ao longo dos anos.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

Abaixo estão sendo selecionados todas as colunas que possuem relação com o acumulado de mudança de temperatura, como também a definição do target.

In [ ]:
relevant_features = ['artificial_surfaces_1000ha', 'forest_area_1000ha', 
                     'carbon_stocks_in_forests_million_tonnes', 'deforestation_rate', 
                     'tree_covered_areas_1000ha']  
target = 'cumulative_temp_change'

In [ ]:
brazil_data = new_data[new_data['country'] == 'Brazil']
brazil_data

In [ ]:
features = [col for col in relevant_features if col in brazil_data.columns]

In [ ]:
features

Agora será aplicado o filtro para buscar apenas as features e os targets, como também dropar dados faltantes.

In [ ]:
# Filtrar dados e remover valores nulos
data_brazil_filtered = brazil_data[features + [target]].dropna()
data_brazil_filtered

Agora serão separados em variaveis independentes e variaveis alvo, e em seguida, treino e teste.

In [ ]:
# Separar X (variáveis independentes) e y (variável alvo)
X = data_brazil_filtered[features]
y = data_brazil_filtered[target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Como forma de avaliar o melhor modelo, será selecionado um grupo de regressores, uma vez que a variavel a ser predita é numérica, como também parametros dos modelos. Em seguida, será aplicado um gridSearch para encontrar os melhores parâmetros e testar. Com isso, os resultados serão comparados para definir o melhor modelo.

In [ ]:
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "RandomForest": RandomForestRegressor(),
    "GradientBoosting": GradientBoostingRegressor()
}

In [ ]:
# Parâmetros de Grid Search
param_grids = {
    "LinearRegression": {},
    "Ridge": {'model__alpha': [0.1, 1.0, 10.0]},
    "Lasso": {'model__alpha': [0.01, 0.1, 1.0]},
    "RandomForest": {'model__n_estimators': [50, 100, 200], 'model__max_depth': [None, 10, 20]},
    "GradientBoosting": {'model__n_estimators': [50, 100, 200], 'model__learning_rate': [0.01, 0.1, 0.5]}
}

In [ ]:
# Pipeline base
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('model', LinearRegression())  # Placeholder
])

In [ ]:
results = {}
for name, model in models.items():
    pipeline.set_params(model=model)
    param_grid = param_grids[name]
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    
    # Melhor modelo e avaliação
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {'BestModel': best_model, 'MSE': mse, 'R2': r2}

In [ ]:
results_df = pd.DataFrame({name: data for name, data in results.items()}).T
print(results_df)

In [ ]:
# Gráfico de comparação de R2
plt.figure(figsize=(8, 5))
r2_scores = results_df['R2']

# Configurar barras e escala do eixo Y
plt.bar(r2_scores.index, r2_scores.values, color='teal')
plt.title('Comparação de R2 entre Modelos')
plt.ylabel('R2 Score')
plt.xticks(rotation=45)

# Ajustar o limite do eixo Y para focar nos valores
min_r2 = r2_scores.min() - 0.01  # Um pouco abaixo do mínimo
max_r2 = r2_scores.max() + 0.01  # Um pouco acima do máximo
plt.ylim(min_r2, max_r2)
plt.grid()
plt.show()

Pelo resultado das comparações, o melhor modelo foi o RandomForest, mesmo que quando comparado o R2, seja pouca a diferença.

In [ ]:
# Melhor modelo - Gráfico de Valores Reais vs Previstos
best_model_name = max(results, key=lambda x: results[x]['R2'])
best_model = results[best_model_name]['BestModel']
y_pred_best = best_model.predict(X_test)

In [ ]:
plt.figure(figsize=(8, 8))
sns.scatterplot(x=y_test, y=y_pred_best, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', lw=2)
plt.title(f'Valores Reais vs Previstos - {best_model_name}')
plt.xlabel('Valores Reais')
plt.ylabel('Valores Previstos')
plt.grid()
plt.show()

Acima temos um gráfico que representa a comparação do que são os dados reais e o que foi obtido. Por mais que a quantidade de dados disponível seja baixa, é possível perceber que o modelo conseguiu descrever o comportamento esperado dos dados. Com um modelo assim, através das variáveis selecionadas seria possível por exemplo tentar predizer o comportamento da temperatura no brasil nos próximos anos.

# Conclusões

Através desse projeto, foi utilizado bases de dados ambientes relevantes para responder perguntas atuais, como países e seus comportamentos de temperatura, desastres naturais e etc. Além disso, na última etapa, ficou claro como pode ser utilizado dados para prever comportamentos como feito com a temperatura do Brazil.

Foi importante observar que os dados dependem de um contexto, olhar globalmente estes dados em grande parte dos objetivos não faz sentido, pois muitos comportamentos ambientais são locais e sentidos apenas nos países afetados, como desastres climáticos. 